## Profiling Tool for the RAPIDS Accelerator for Apache Spark

To run the profiling tool, enter the log path that represents the location of your Spark GPU event logs. Then, select "Run all" to execute the notebook. Once the notebook completes, various output tables will appear below. For more options on running the profiling tool, please refer to the [Profiling Tool User Guide](https://docs.nvidia.com/spark-rapids/user-guide/latest/profiling/quickstart.html#running-the-tool).

### Note
- Currently, local and S3 event log paths are supported.
- Eventlog path must follow the formats `/local/path/to/eventlog` for local logs or `s3://my-bucket/path/to/eventlog` for logs stored in S3.
- The specified path can also be a directory. In such cases, the tool will recursively search for event logs within the directory.
   - For example: `/path/to/clusterlogs`
- To specify multiple event logs, separate the paths with commas.
   - For example: `s3://my-bucket/path/to/eventlog1,s3://my-bucket/path/to/eventlog2`

## User Input

In [ ]:
# Path to the event log in S3 (or local path)
EVENTLOG_PATH = "s3://my-bucket/path/to/eventlog"  # or "/local/path/to/eventlog"

# S3 path with write access where the output will be copied. 
S3_OUTPUT_PATH = "s3://my-bucket/path/to/output"

## Setup Environment

In [ ]:
from IPython.display import display, Markdown

TOOLS_VER = "24.08.2"
display(Markdown(f"**Using Spark RAPIDS Tools Version:** {TOOLS_VER}"))

In [ ]:
%pip install spark-rapids-user-tools==$TOOLS_VER --user > /dev/null 2>&1

In [ ]:
import os
import pandas as pd

# Update PATH to include local binaries
os.environ['PATH'] += os.pathsep + os.path.expanduser("~/.local/bin")

OUTPUT_PATH = "/tmp"
DEST_FOLDER_NAME = "prof-tool-result"

# Set environment variables
os.environ["EVENTLOG_PATH"] = EVENTLOG_PATH 
os.environ["OUTPUT_PATH"] = OUTPUT_PATH

CONSOLE_OUTPUT_PATH = os.path.join(OUTPUT_PATH, 'console_output.log')
CONSOLE_ERROR_PATH = os.path.join(OUTPUT_PATH, 'console_error.log')

os.environ['CONSOLE_OUTPUT_PATH'] = CONSOLE_OUTPUT_PATH
os.environ['CONSOLE_ERROR_PATH'] = CONSOLE_ERROR_PATH

print(f'Console output will be stored at {CONSOLE_OUTPUT_PATH} and errors will be stored at {CONSOLE_ERROR_PATH}')


## Run Profiling Tool

In [ ]:
%%sh
spark_rapids profiling --platform emr --eventlogs "$EVENTLOG_PATH" -o "$OUTPUT_PATH" --verbose > "$CONSOLE_OUTPUT_PATH" 2> "$CONSOLE_ERROR_PATH"

## Console Output
Console output shows the top candidates and their estimated GPU speedup.


In [ ]:
%%sh
cat $CONSOLE_OUTPUT_PATH

In [ ]:
%%sh
cat $CONSOLE_ERROR_PATH

In [ ]:
import re
import shutil
import os


def extract_file_info(console_output_path, output_base_path):
    try:
        with open(console_output_path, 'r') as file:
            stdout_text = file.read()

        # Extract log file location
        location_match = re.search(r"Location: (.+)", stdout_text)
        if not location_match:
            raise ValueError(
                "Log file location not found in the provided text.")

        log_file_location = location_match.group(1)

        # Extract profiling output folder
        qual_match = re.search(r"prof_[^/]+(?=\.log)", log_file_location)
        if not qual_match:
            raise ValueError(
                "Output folder not found in the log file location.")

        output_folder_name = qual_match.group(0)
        output_folder = os.path.join(output_base_path, output_folder_name)
        return output_folder, log_file_location

    except Exception as e:
        raise RuntimeError(f"Cannot parse console output. Reason: {e}")


def copy_logs(destination_folder, *log_files):
    try:
        log_folder = os.path.join(destination_folder, "logs")
        os.makedirs(log_folder, exist_ok=True)

        for log_file in log_files:
            if os.path.exists(log_file):
                shutil.copy2(log_file, log_folder)
            else:
                print(f"Log file not found: {log_file}")
    except Exception as e:
        raise RuntimeError(f"Cannot copy logs to output. Reason: {e}")


try:
    output_folder, log_file_location = extract_file_info(
        CONSOLE_OUTPUT_PATH, OUTPUT_PATH)
    jar_output_folder = os.path.join(output_folder,
                                     "rapids_4_spark_profile")
    print(f"Output folder detected {output_folder}")
    copy_logs(output_folder, log_file_location, CONSOLE_OUTPUT_PATH,
              CONSOLE_ERROR_PATH)
    print(f"Logs successfully copied to {output_folder}")
except Exception as e:
    print(e)

## Download Output

In [ ]:
import shutil
import os
import subprocess
from IPython.display import HTML, display
from urllib.parse import urlparse

def display_error_message(error_message, exception):
    error_message_html = f"""
    <div style="color: red; margin: 20px;">
        <strong>Error:</strong> {error_message}.
        <br/>
        <strong>Exception:</strong> {exception}
    </div>
    """
    display(HTML(error_message_html))

def copy_file_to_s3(local_file: str, bucket: str, destination_folder_name: str):
    try:
        file_name = os.path.basename(local_file)
        s3_path = f"s3://{bucket}/{destination_folder_name}/{file_name}"
        subprocess.run(["aws", "s3", "cp", local_file, s3_path], check=True, capture_output=True, text=True)
        return construct_download_url(file_name, bucket, destination_folder_name)
    except subprocess.CalledProcessError as e:
        raise Exception(f"Error copying file to S3: {e.stderr}") from e

def get_default_aws_region():
    try:
        return subprocess.check_output(
            "aws configure list | grep region | awk '{print $2}'",
            shell=True,
            text=True
        ).strip()
    except subprocess.CalledProcessError:
        return "Error: Unable to retrieve the region."

def construct_download_url(file_name: str, bucket_name: str, destination_folder_name: str):
    region = get_default_aws_region()
    return f"https://{region}.console.aws.amazon.com/s3/object/{bucket_name}?region={region}&prefix={destination_folder_name}/{file_name}"

def create_download_link(source_folder, bucket_name, destination_folder_name):
    folder_to_compress = os.path.join("/tmp", os.path.basename(source_folder))
    local_zip_file_path = shutil.make_archive(folder_to_compress, 'zip', source_folder)
    download_url = copy_file_to_s3(local_zip_file_path, bucket_name, destination_folder_name)

    download_button_html = f"""
    <style>
        .download-btn {{
            display: inline-block;
            padding: 10px 20px;
            font-size: 16px;
            color: white;
            background-color: #4CAF50;
            text-align: center;
            text-decoration: none;
            border-radius: 5px;
            border: none;
            cursor: pointer;
            margin: 15px auto;
        }}
        .download-btn:hover {{
            background-color: #45a049;
        }}
        .button-container {{
            display: flex;
            justify-content: center;
            align-items: center;
        }}
        .button-container a {{
            color: white !important;
        }}
    </style>

    <div style="color: #444; font-size: 14px; text-align: center; margin: 10px;">
        Zipped output file created at {download_url}
    </div>
    <div class='button-container'>
        <a href='{download_url}' class='download-btn'>Download Output</a>
    </div>
    """
    display(HTML(download_button_html))

try:
    current_working_directory = os.getcwd()
    parsed_s3_output_path = urlparse(S3_OUTPUT_PATH)
    bucket_name = parsed_s3_output_path.netloc
    destination_path = os.path.join(parsed_s3_output_path.path.strip("/"), DEST_FOLDER_NAME.strip("/"))
    create_download_link(output_folder, bucket_name, destination_path)
    
except Exception as e:
    error_msg = f"Failed to create download link for {output_folder}"
    display_error_message(error_msg, e)


## Application Status

The report show the status of each eventlog file that was provided


In [ ]:
try:
    status_output = pd.read_csv(
        os.path.join(jar_output_folder, "profiling_status.csv"))

    # Set options to display the full content of the DataFrame
    pd.set_option('display.max_rows', None)  # Show all rows
    pd.set_option('display.max_columns', None)  # Show all columns
    pd.set_option('display.width', None)  # Adjust column width to fit the display
    pd.set_option('display.max_colwidth', None)  # Display full content of each column

    display(status_output)
except Exception as e:
    error_msg = "Unable to show Application Status"
    display_error_message(error_msg, e)        
        
        

## GPU Job Tuning Recommendations
This has general suggestions for tuning your applications to run optimally on GPUs.


In [ ]:
try:
    jar_output_folder = os.path.join(output_folder, "rapids_4_spark_profile")
    app_df = pd.DataFrame(columns=['appId', 'appName'])

    for x in os.scandir(jar_output_folder):
        if x.is_dir():
            csv_path = os.path.join(x.path, "application_information.csv")
            if os.path.exists(csv_path):
              tmp_df = pd.read_csv(csv_path)
              app_df = pd.concat([app_df, tmp_df[['appId', 'appName']]])


    app_list = app_df["appId"].tolist()
    app_recommendations = pd.DataFrame(columns=['App', 'Recommended Configuration'])

    for app in app_list:
      app_file = open(os.path.join(jar_output_folder, app, "profile.log"))
      recommendations_start = 0
      recommendations_str = ""
      for line in app_file:
        if recommendations_start == 1:
          recommendations_str = recommendations_str + line
        if "### D. Recommended Configuration ###" in line:
          recommendations_start = 1
      app_recommendations = pd.concat([app_recommendations, pd.DataFrame({'App': [app], 'Recommended Configuration': [recommendations_str]})], ignore_index=True)
      html = app_recommendations.to_html().replace("\\n", "<br>")
      style = "<style>table td { vertical-align: top !important; text-align: left !important; white-space: pre-wrap; } th { text-align: left !important; }</style>"
      display(HTML(html + style))
except Exception as e:
    error_msg = "Unable to show stage output"
    display_error_message(error_msg, e) 